In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import random
import gym
import cv2
import math
from PIL import Image
import pandas as pd
from scipy.stats import norm
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import clip

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
prompts = [
        f"Create an image of a working on a tour plan in a",
        f"Create an image of a brainstorming new ideas in a",
        f"Create an image of a actively working on a project in a",
        f"Create an image of a reflecting on their work in a",
        f"Create an image of a collaborating with colleagues in a",
        f"Create an image of a teaching or presenting in a",
        f"Create an image of a conducting research in a",
        f"Create an image of a creating an art piece in a",
        f"Create an image of a solving a complex problem in a",
        f"Create an image of a giving a speech or a lecture in a",
        f"Create an image of a experimenting with new techniques in a"
        f"Create an image of a designing a new invention in a",
        f"Create an image of a leading a team meeting in a",
        f"Create an image of a analyzing data on a computer in a",
        f"Create an image of a writing a book in a",
        f"Create an image of a gardening in a",
        f"Create an image of a playing a musical instrument in a",
        f"Create an image of a practicing yoga in a",
        f"Create an image of a cooking in a gourmet kitchen in a",
        f"Create an image of a building a robot in a",
        f"Create an image of a exploring a historic site in a"
    ]

def get_prompt(random):
    prompts = [
        f"Create an image of a working on a tour plan in a",
        f"Create an image of a brainstorming new ideas in a",
        f"Create an image of a actively working on a project in a",
        f"Create an image of a reflecting on their work in a",
        f"Create an image of a collaborating with colleagues in a",
        f"Create an image of a teaching or presenting in a",
        f"Create an image of a conducting research in a",
        f"Create an image of a creating an art piece in a",
        f"Create an image of a solving a complex problem in a",
        f"Create an image of a giving a speech or a lecture in a",
        f"Create an image of a experimenting with new techniques in a"
        f"Create an image of a designing a new invention in a",
        f"Create an image of a leading a team meeting in a",
        f"Create an image of a analyzing data on a computer in a",
        f"Create an image of a writing a book in a",
        f"Create an image of a gardening in a",
        f"Create an image of a playing a musical instrument in a",
        f"Create an image of a practicing yoga in a",
        f"Create an image of a cooking in a gourmet kitchen in a",
        f"Create an image of a building a robot in a",
        f"Create an image of a exploring a historic site in a"
    ]
    return prompts[random]

descriptions = ['unique', 'distinctive', 'cool']  
professions = ['scientist', 'artist', 'professor']  
settings = ['corporate office', 'research center', 'classroom'] 

def create_prompt(a,b,c, prompt):
    sentence = prompt[:21] + descriptions[a] + " " + professions[b] + " " + prompt[21:] + ' ' +settings[c]+ '.'
    return sentence


In [ ]:
NUM_ACTION = 3


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)



action_list = {}
action_total = {}
count = 0
for i in range(5):
    for j in range(5):
        for k in range(5):
            action_list[count] = 0
            action_total[count] = 0
            count += 1

action_dict = {}
count = 0
for i in range(NUM_ACTION):
    for j in range(NUM_ACTION):
        for k in range(NUM_ACTION):
            action_dict[count] = (i, j, k)
            count += 1

## Random

In [ ]:
for i in range(500):
    prompt = get_prompt(random.randint(0, 19))
    action = random.randint(0,20)

    word1, word2, word3 = action_dict[action]
    current_word = create_prompt(word1,word2,word3,prompt)

    prediction = pipe([current_word]).images

    clip_text = clip.tokenize([current_word]).to(device)
        
    clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
    with torch.no_grad():
        image_features = clip_model.encode_image(clip_image)
        text_features = clip_model.encode_text(clip_text)

    cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
    print(f'episode: {i}, similar : {cosine_similarity}')
    if cosine_similarity < 0.3:
        action_list[action] += 1
    action_total[action] +=1   
    

In [ ]:
print(action_list)
print(action_total)

## Epsilon 0.01

In [ ]:
epsilon = 0.01
prob = [1]*20
action_list = {}
action_total = {}
count = 0
for i in range(5):
    for j in range(5):
        for k in range(5):
            action_list[count] = 0
            action_total[count] = 0
            count += 1
for i in range(500):
    prompt_index  = random.choices(range(len(prompts)), prob,k=1)
    prompt = prompts[prompt_index[0]]
    action = random.randint(0,20)

    word1, word2, word3 = action_dict[action]
    current_word = create_prompt(word1,word2,word3,prompt)

    prediction = pipe([current_word]).images

    clip_text = clip.tokenize([current_word]).to(device)
        
    clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
    with torch.no_grad():
        image_features = clip_model.encode_image(clip_image)
        text_features = clip_model.encode_text(clip_text)

    cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
    print(f'episode: {i}, similar : {cosine_similarity}')
    if cosine_similarity < 0.3:
        action_list[action] += 1
        prob[prompt_index[0]]+=epsilon
    action_total[action] +=1   
    

In [ ]:
print(action_list)
print(action_total)

## Epsilon 0.1

In [ ]:
epsilon = 0.1
prob = [1]*20
action_list = {}
action_total = {}
count = 0
for i in range(5):
    for j in range(5):
        for k in range(5):
            action_list[count] = 0
            action_total[count] = 0
            count += 1
for i in range(500):
    prompt_index = random.choices(range(len(prompts)), prob,k=1)
    prompt = prompts[prompt_index[0]]
    action = random.randint(0,20)

    word1, word2, word3 = action_dict[action]
    current_word = create_prompt(word1,word2,word3,prompt)

    prediction = pipe([current_word]).images

    clip_text = clip.tokenize([current_word]).to(device)
        
    clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
    with torch.no_grad():
        image_features = clip_model.encode_image(clip_image)
        text_features = clip_model.encode_text(clip_text)

    cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
    print(f'episode: {i}, similar : {cosine_similarity}')
    if cosine_similarity < 0.3:
        action_list[action] += 1
        prob[prompt_index[0]]+=epsilon
    action_total[action] +=1   
    

In [ ]:
print(action_list)
print(action_total)

## Epsilon 0.5

In [ ]:
epsilon = 0.5
prob = [1]*20
action_list = {}
action_total = {}
count = 0
for i in range(5):
    for j in range(5):
        for k in range(5):
            action_list[count] = 0
            action_total[count] = 0
            count += 1
for i in range(500):
    prompt_index = random.choices(range(len(prompts)), prob,k=1)
    prompt = prompts[prompt_index[0]]
    action = random.randint(0,20)

    word1, word2, word3 = action_dict[action]
    current_word = create_prompt(word1,word2,word3,prompt)

    prediction = pipe([current_word]).images

    clip_text = clip.tokenize([current_word]).to(device)
        
    clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
    with torch.no_grad():
        image_features = clip_model.encode_image(clip_image)
        text_features = clip_model.encode_text(clip_text)

    cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
    print(f'episode: {i}, similar : {cosine_similarity}')
    if cosine_similarity < 0.3:
        action_list[action] += 1
        prob[prompt_index[0]]+=epsilon
    action_total[action] +=1   
    

In [ ]:
print(action_list)
print(action_total)

## Threshold

In [ ]:
action_list = {}
action_total = {}
count = 0
thres = 0
prompt = get_prompt(random.randint(0, 19))
for i in range(5):
    for j in range(5):
        for k in range(5):
            action_list[count] = 0
            action_total[count] = 0
            count += 1

for i in range(500):
    
    action = random.randint(0,20)

    if thres == 5:
        prompt = get_prompt(random.randint(0, 19))
        
    word1, word2, word3 = action_dict[action]
    current_word = create_prompt(word1,word2,word3,prompt)

    prediction = pipe([current_word]).images

    clip_text = clip.tokenize([current_word]).to(device)
        
    clip_image = clip_preprocess(prediction[0]).unsqueeze(0).to(device)
        
    with torch.no_grad():
        image_features = clip_model.encode_image(clip_image)
        text_features = clip_model.encode_text(clip_text)

    cosine_similarity = torch.nn.functional.cosine_similarity(text_features, image_features, dim=1)
    print(f'episode: {i}, similar : {cosine_similarity}')
    if cosine_similarity < 0.3:
        action_list[action] += 1
    else:
        thres +=1
    action_total[action] +=1   
    

In [ ]:
print(action_list)
print(action_total)